In [18]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [19]:
tsp_data_txt = open('tsp_dataset.txt')

In [20]:
lines = tsp_data_txt.readlines()

In [21]:
tsp_data_txt = open('tsp_dataset.txt')
data = {}
lines = tsp_data_txt.readlines()
distance_matrix = np.zeros((len(lines),len(lines)))
for i,line in enumerate(lines):
    for j,element in enumerate((line.split('\n')[0].split('       '))):
        if(element == ''):
            continue
        
        distance_matrix[i][j-1] = int(element.strip())
data['distance_matrix']= distance_matrix
data['reward_matrix'] = [1 for i  in range(0,len(distance_matrix))]
data['depot'] = 0
data['n_state'] = sum(list(range(1,len(distance_matrix))))
data['n_action'] = len(distance_matrix)
data['visited_nodes'] = []


In [22]:
for i in range(0,len(data['distance_matrix'])):
    data['distance_matrix'][i][i] = 9999


In [23]:
distance_matrix[1][2]

55.0

In [24]:
def change_reward(envp):
    for i in range(0,envp['n_state']):
        for j in range(0,envp['n_action']):
            for k in range(0,envp['n_action']):
                if k==0:
                    env['P'][i][j][k][2]= 10
                    env['P'][i][j][k][3]= True
                    break
    return envp

In [25]:
def initialze_value(env):
    initial_P = 1/(env['n_action']-1)
    env['P'] = {}
    
    #P[state][action] = [(prob, next state, reward, done), ...]
    
    for i in range(0,env['n_state']):
        env['P'][i] ={}
        
        if(i==104):
            for j in range(0,env['n_action']):
                env['P'][i][j] = []
                initial_P = 1/(env['n_action']-1)
                for k in range(0,env['n_action']):
                    if (j == env['depot'] and k == env['depot']):
                        ini_temp = [1,k,10,True]
                        env['P'][i][j].append(ini_temp)
                        initial_P = 0
                        continue
                    ini_temp = [initial_P,k,data['reward_matrix'][k],False]
                    env['P'][i][j].append(ini_temp)
        else:
            for j in range(0,env['n_action']):
                env['P'][i][j] = []
                for k in range(0,env['n_action']):
                    ini_temp = [initial_P,k,data['reward_matrix'][k],False]
                    env['P'][i][j].append(ini_temp)
    
    env['state'] = []
    for si in range(0,env['n_state']):
        env['state'].append([])
        for n in range(0,env['n_action']):
            env['state'][si].append(initial_P)
    
    env['state'] = np.asarray(env['state']).reshape((env['n_action'],-1))
            
    return env
    

In [26]:
# Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

# Run configurations
print_every = 100
num_episodes = 100000

In [27]:
class REINFORCE(nn.Module):
    def __init__(self):
        super(REINFORCE, self).__init__()
        self.linear_1 = nn.Linear(105, 128)
        self.linear_2 = nn.Linear(128, 15)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.data = []
        self.visited_list =[]
    
    def get_action_with_softmax(self,prob):
        visited_index = []
        prob[0][0] =  torch.tensor(float('-inf'))
        first_index = Categorical(F.softmax(prob[0])).sample()
        visited_index.append(first_index)
        for pi,pb in enumerate(prob[1:]):    
            if pi <13:
                prob[pi+1][0] =  torch.tensor(float('-inf'))
            for vi,viv in enumerate(visited_index):
                prob[pi+1][viv] =  torch.tensor(float('-inf'))
            temp_index = Categorical(F.softmax(prob[pi+1])).sample()
            visited_index.append(temp_index)
        return torch.tensor(visited_index),F.softmax(prob)

        
    def policy(self, state):
        x = F.relu(self.linear_1(state))
        x = self.linear_2(x)
        return x
    

    def get_action(self, state):
        prob = self.policy(torch.from_numpy(state).float())
        return self.get_action_with_softmax(prob)
      
    def save(self, item):
        self.data.append(item)
        
    def update(self):
        G = 0
        self.optimizer.zero_grad()
        for reward, prob in self.data[::-1]:
            G = reward
            loss = torch.tensor(reinforce.get_reward(action), requires_grad=True)
            loss.backward()
        self.optimizer.step()
        self.data = []
        
    def get_reward(self,visit_list):
        depot =0
        distance = torch.tensor(data['distance_matrix'])[0][visit_list[0]]
        prev_cite = visit_list[0]
        for i in range(1,14):
            distance+= torch.tensor(data['distance_matrix'])[prev_cite][visit_list[i]]
            prev_cite =visit_list[i]

        distance+= torch.tensor(data['distance_matrix'])[prev_cite][visit_list[14]]
        return distance

In [28]:
env = initialze_value(data)

In [29]:
def get_prob_for_update(action,prob):
    prob_list = torch.zeros((15,15))
    for pi,pl in enumerate(prob_list):
        prob_list[pi]+= prob[action[pi]]
    return prob_list
    

In [30]:
def train(agent, env):
    G = 0.0
    for n_epi in range(num_episodes):
        state = env['state']
        action, prob = agent.get_action(state)
        reward = agent.get_reward(action)
        agent.save((reward, prob))
        G += reward
            
        agent.update()
        if n_epi % print_every==0 and n_epi!=0:
            avg_return = G / print_every
            print("# of episode :{}, avg score : {}".format(n_epi, avg_return))
            G = 0.0
            if avg_return <= 650:
                break

In [31]:
reinforce = REINFORCE()
state =env['state']
       
action, prob = reinforce.get_action(state)
torch.tensor(reinforce.get_reward(action), requires_grad=True).backward()

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  first_index = Categorical(F.softmax(prob[0])).sample()
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  temp_index = Categorical(F.softmax(prob[pi+1])).sample()
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.tensor(visited_index),F.softmax(prob)
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/2100599326.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sour

In [32]:
zzz = get_prob_for_update(action,prob)

In [33]:
train(reinforce, env)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  first_index = Categorical(F.softmax(prob[0])).sample()
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  temp_index = Categorical(F.softmax(prob[pi+1])).sample()
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.tensor(visited_index),F.softmax(prob)
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sou

# of episode :100, avg score : 664.95
# of episode :200, avg score : 657.54
# of episode :300, avg score : 671.07
# of episode :400, avg score : 670.82
# of episode :500, avg score : 661.35
# of episode :600, avg score : 663.58
# of episode :700, avg score : 668.38
# of episode :800, avg score : 658.51
# of episode :900, avg score : 673.74
# of episode :1000, avg score : 661.07
# of episode :1100, avg score : 656.98
# of episode :1200, avg score : 666.61
# of episode :1300, avg score : 666.42
# of episode :1400, avg score : 655.55
# of episode :1500, avg score : 660.58
# of episode :1600, avg score : 656.83
# of episode :1700, avg score : 662.92
# of episode :1800, avg score : 652.54
# of episode :1900, avg score : 665.13
# of episode :2000, avg score : 658.27
# of episode :2100, avg score : 673.56
# of episode :2200, avg score : 662.84
# of episode :2300, avg score : 674.42
# of episode :2400, avg score : 654.6
# of episode :2500, avg score : 669.55
# of episode :2600, avg score : 658

In [34]:
zzz =get_prob_for_update(action,prob)

In [35]:
action, prob = reinforce.get_action(state)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  first_index = Categorical(F.softmax(prob[0])).sample()
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  temp_index = Categorical(F.softmax(prob[pi+1])).sample()
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_18648/4081901389.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.tensor(visited_index),F.softmax(prob)


In [36]:
action

tensor([ 6,  5, 13, 11, 14,  2, 10,  3,  7,  9,  4,  1, 12,  8,  0])

In [37]:
reinforce.get_reward(action)

tensor(604., dtype=torch.float64)

In [48]:
[int(a) for a in action]

[6, 5, 13, 11, 14, 2, 10, 3, 7, 9, 4, 1, 12, 8, 0]

In [39]:
def get_reward(visit_list):
    depot =0
    distance = (data['distance_matrix'])[0][visit_list[0]]
    prev_cite = visit_list[0]
    for i in range(1,14):
        distance+= (data['distance_matrix'])[prev_cite][visit_list[i]]
        prev_cite =visit_list[i]

    distance+= (data['distance_matrix'])[prev_cite][visit_list[14]]
    return distance

In [40]:
bsline

[12, 1, 14, 8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0]

In [49]:
get_reward([6, 5, 13, 11, 14, 2, 10, 3, 7, 9, 4, 1, 12, 8, 0])

604.0